# Derivation of maximum-dissipation based GSM


In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()
from bmcs_utils.api import Cymbol, cymbols

## R_g - Generic Lagrangian augmented with consistency 

### Helmholtz potential and its rate

In [ ]:
class R_g:
    # time
    t = Cymbol(r't', codename='t')
    # state variables
    eps = sp.Function(r'\varepsilon', codename='eps')(t)
    Eps = sp.Function(r'\boldsymbol{\mathcal{E}}', codename='Eps')(t)
    # time derivatives
    dot_eps_ = eps.diff(t)
    dot_Eps_ = Eps.diff(t)

    Sig = sp.Function(r'\boldsymbol{\mathcal{S}}', codename='Sig')(eps, Eps)
    dot_Sig_ = Sig.diff(t)

    # Helmholtz free energy
    psi_ = sp.Function('psi')(eps, Eps)

    # dissipation potential
    pi_mech = sp.Function(r'{\pi_\mathrm{mech}}')(Eps, dot_Eps_)

    # threshold function for elastic domain
    f_ = sp.Function(r'{f}')(Eps, Sig)

    # # flow potential
    g_ = sp.Function(r'{g}')(Eps, Sig)

    dot_psi_ = psi_.diff(t)

    # mechanical dissipation
    gamma_mech_ = -psi_.diff(Eps) * dot_Eps_ # - pi_mech

    # expressions for thermodynamic forces as gradient of the Helmholtz free energy
    Sig_ = psi_.diff(Eps)

    # Lagrangian and its gradient
    dot_lam = Cymbol(r'\dot{\lambda}', codename='dot_lambda')
    L_ = -gamma_mech_ + dot_lam * g_
    dL_dEps_ = L_.diff(Eps)

    # flow potential and its rate
    dot_f_ = f_.diff(t)

    R_ = sp.Matrix([dL_dEps_, dot_f_])



In [ ]:
R_g.pi_mech

Helmholtz free energy and its rate

In [ ]:
R_g.psi_, R_g.dot_psi_

Mechanical dissipation as a negative rate of change of the Helmholtz free energy with respect to the rate of internal variables

In [ ]:
R_g.gamma_mech_

Thermodynamic foces and their rates as symbols

In [ ]:
R_g.Sig, R_g.dot_Sig_

Dissipation function is the negative Helmholtz rate

Threshold function defined in terms of $\mathcal{S}(\varepsilon, \mathcal{E})$ 

In [ ]:
R_g.f_

### Lagrangian and its gradient

In [ ]:
R_g.L_

In [ ]:
R_g.dL_dEps_

### Consistency criterion - zero rate of flow potential

In [ ]:
R_g.dot_f_

### Residuum function $\mathcal{R}$

In [ ]:
R_g.R_

In [ ]:
sp.simplify(sp.solve(R_g.R_, [R_g.dot_Eps_, R_g.dot_lam]))[R_g.dot_Eps_]

In [ ]:
sp.solve(R_g.R_[0], R_g.dot_Eps_)[0]

## R_ig - General incremental Lagrangian

In [ ]:
class R_ig:

    # time
    delta_t = Cymbol(r'\Delta t', codename='delta_t')
    dot_lam = Cymbol(r'\dot{\lambda}', codename='dot_lam')
    
    # fundamental state
    Eps_n = Cymbol(r'\boldsymbol{\mathcal{E}}_n', codename='Eps_n')
    eps_n = Cymbol(r'\varepsilon_n', codename='eps_n')

    # increment
    delta_Eps_n = Cymbol(r'\Delta{\boldsymbol{\mathcal{E}}}_n', codename='delta_Eps_n')
    delta_eps_n = Cymbol(r'\Delta{\varepsilon}_n', codename='delta_eps_n')
    delta_lam_n = Cymbol(r'\Delta{\lambda}_n', codename='delta_lam_n')

    # updated state
    Eps_n1 = Eps_n + delta_Eps_n
    eps_n1 = eps_n + delta_eps_n

    # rate of change
    dot_Eps_n = delta_Eps_n / delta_t
    dot_eps_n = delta_eps_n / delta_t
    dot_lam_n = delta_lam_n / delta_t

    # derive substitutions
    dot_Eps = R_g.Eps.diff(R_g.t)
    dot_eps = R_g.eps.diff(R_g.t)
    dot_lam = dot_lam

    # substitute the time derivatives
    subs_dot_Eps = {dot_Eps: dot_Eps_n}
    subs_dot_eps = {dot_eps: dot_eps_n}
    subs_dot_lam = {dot_lam: dot_lam_n}
    subs_Eps = {R_g.Eps: Eps_n1}
    subs_eps = {R_g.eps: eps_n1}
    subs_n1 = {**subs_dot_Eps, **subs_dot_eps, **subs_dot_lam, **subs_Eps, **subs_eps}

    # residuum vector in n+1 step
    R_n1 = R_g.R_.subs(subs_n1)

    # construct the jacobian of the residuum
    delta_A_ig_ = sp.Matrix([delta_Eps_n, delta_lam_n])
    dR_dA_n1 = R_n1.jacobian(delta_A_ig_)

R_ig.subs_n1

In [ ]:
R_ig.R_n1

In [ ]:
sp.cse(R_ig.R_n1)

### Construct the Jacobian

In [ ]:
R_ig.delta_A_ig_

In [ ]:
R_ig.dR_dA_n1

In [ ]:
dR_dA_ig_cse_ = sp.cse(R_ig.dR_dA_n1, optimizations='basic')
dR_dA_ig_cse_[0]

In [ ]:
dR_dA_ig_cse_[1][0]

## R_g[.] - Parameterized residuum derivation

$$
\mathcal{L}(\varepsilon, \boldsymbol{\mathcal{E}},
\dot{\boldsymbol{\mathcal{E}}}) = 
- \gamma_\mathrm{mech}(\varepsilon, \boldsymbol{\mathcal{E}}, 
\dot{\boldsymbol{\mathcal{E}}})
+
\dot{\lambda} g(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}}) 
$$

Noting that the lagrangian combines not only the values of the internal variables but also their rates
the structure of the solver takes a slightly different form compared to the standard constrained 
optimization problem which is fully defined by introducing the Kuhn-Tucker conditions and constructing 
the Lagrangian. Since the unknowns in our optimization problem are not the internal variables but their rates, the Kuhn-Tucker conditions are not sufficient to solve the problem. Instead, we need to introduce the
rate of the Lagrange multiplier as an additional unknown. The rate of the Lagrange multiplier is
determined by employing the consistency condition ensuring that the rate of the Lagrange multiplier
is such that the constraint is implicitly satisfied. In other words, the rate of the Lagrange multiplier
is determined by the requirement that the constraint is satisfied at the rate level. This is the key
difference between the standard constrained optimization problem and the problem of the rate-based
formulation of the constitutive model.

The generic residuum that we are going to derive 
from the three potentials $\psi$, $f$, and $g$ will have the form:

\begin{align}
\mathcal{R}_{\mathrm{g}} = \left[
\begin{array}{c}
\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L}
(\varepsilon, \boldsymbol{\mathcal{E}}, \dot{\boldsymbol{\mathcal{E}}}, \dot{\lambda}) \\
\dot{f}(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}})
\end{array}
\right]
\end{align}

### Gradient of the Lagrangian

Print the generic form of the Lagrangian gradient $\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L}$ as  the first component of the residuum $\mathcal{R}_\mathrm{g}$. The abstract representation provides a guidance for the construction of the elastoplastic solver $\mathcal{R}_\mathrm{ep}$ starting from the Helmholtz energy $\psi_\mathrm{ep}$ and threshold function $f_\mathrm{ep}$ with the associated flow potential $g_\mathrm{ep} = f_\mathrm{ep}$  

This can rephrased as 
$$
\nabla^2_{\!\boldsymbol{\mathcal{E}}} \psi(\varepsilon, \boldsymbol{\mathcal{E}})
\cdot \dot{\boldsymbol{\mathcal{E}}}
+
\left(
\nabla_{\!\boldsymbol{\mathcal{S}}}f
\cdot
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
+
\nabla_{\!\boldsymbol{\mathcal{E}}}f
\right)
\dot{\lambda} = 0
$$

Note that the threshold function is formulated in terms of thermodynamic forces $\boldsymbol{\mathcal{S}}$ may need to align the sign with the conjugate internal variable. This configurational feature is introduced using the diagnoal matrix $\Upsilon$ containing either values 1 or -1. 
$$
 \boldsymbol{\mathcal{S}} = \boldsymbol{\Upsilon} \nabla_{\!\boldsymbol{\mathcal{E}}} \psi.
$$

### Rate of the flow potential

The general form of the rate of the flow potential is given by the following expression: 
$$
 \dot{g}(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}}) =
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}} +
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\dot{\boldsymbol{\mathcal{S}}}
$$
where
$$
\dot{\boldsymbol{\mathcal{S}}} = 
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon} +
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
$$
so that the whole expression reads
$$
 \dot{g}(\boldsymbol{\mathcal{E}}, \boldsymbol{\mathcal{S}}) =
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}}
+
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\left(
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon}
+
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
\right)
= 0.
$$

### Combining the two expressions we obtain the whole residuum

Delivering the residuum as
$$
\mathcal{R}_\mathrm{g} = \left[
\begin{array}{c}
\nabla_{\!\boldsymbol{\mathcal{E}}}\mathcal{L} \\
\dot{g}
\end{array}
\right] = \left[
\begin{array}{c}
\nabla^2_{\!\boldsymbol{\mathcal{E}}} \psi
\cdot \dot{\boldsymbol{\mathcal{E}}} +
\left(
\nabla_{\!\boldsymbol{\mathcal{S}}}f
\cdot
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}} +
\nabla_{\!\boldsymbol{\mathcal{E}}}f
\right)
\dot{\lambda} \\
\nabla_{\!\boldsymbol{\mathcal{E}}}g
\cdot
\dot{\boldsymbol{\mathcal{E}}} +
\nabla_{\!\boldsymbol{\mathcal{S}}}g
\cdot
\left(
\nabla_{\!\varepsilon}\boldsymbol{\mathcal{S}}
\cdot
\dot{\varepsilon} +
\nabla_{\!\boldsymbol{\mathcal{E}}}\boldsymbol{\mathcal{S}}
\cdot
\dot{\boldsymbol{\mathcal{E}}}
\right)
\end{array}
\right] =
\boldsymbol{0}
$$

In [ ]:
def derive_R_g(P, dot_lam):
    """
    Derive the residuum vector combining the gradient of the Lagrangian and the rate of the flow potential.

    Parameters
    ----------
    psi_ : sympy expression
        The free energy potential as a function of internal variables.
    eps : sympy symbol
        The external variable.
    dot_eps : sympy symbol
        The rate of change of the external variable.
    Eps : sympy matrix
        The vector of internal variables.
    dot_Eps : sympy matrix
        The rate of change of the internal variables.
    Sig : sympy matrix
        The conjugate vector of thermodynamic forces.
    Sig_signs : list or array-like
        Array of signs.
    f_ : sympy expression
        The yield function.
    g_ : sympy expression
        The flow potential function.

    Returns
    -------
    sympy matrix
        The residuum vector combining the gradient of the Lagrangian and the rate of the flow potential.

    Notes
    -----
    The derivation involves computing the gradient of the Lagrangian with respect to internal variables
    and the rate of change of the flow potential.
    """
    psi_, eps, dot_eps, Eps, dot_Eps, Sig, Sig_signs, f_, g_ = P.psi_, P.eps, P.dot_eps, P.Eps, P.dot_Eps, P.Sig, P.Sig_signs, P.f_, P.g_
    # Compute the gradient of the Lagrangian
    Sig_ = sp.simplify(sp.diag(*Sig_signs) * psi_.diff(Eps))
#    df_dEps_ = ((f_.diff(Sig).T * Sig_.jacobian(Eps)).T + f_.diff(Eps)).subs(dict(zip(Sig, Sig_)))
    dg_dEps_ = ((g_.diff(Sig).T * Sig_.jacobian(Eps)).T + g_.diff(Eps)).subs(dict(zip(Sig, Sig_)))
#    dg_dEps_ = ((g_.diff(Sig).T * Sig_.jacobian(Eps)).T).subs(dict(zip(Sig, Sig_)))
    dL_dEps_ = sp.hessian(psi_, Eps).T * dot_Eps + dg_dEps_ * dot_lam

    # Compute the rate of the flow potential
    dot_psi_eps_ = Sig_.diff(eps) * dot_eps
    dot_psi_Eps_ = Sig_.jacobian(Eps) * dot_Eps
    dot_Sig_Eps_ = dot_psi_eps_ + dot_psi_Eps_
#    dot_g_ = (g_.diff(Eps).T * dot_Eps + g_.diff(Sig).T * dot_Sig_Eps_).subs(dict(zip(Sig, Sig_)))
    dot_f_ = (f_.diff(Eps).T * dot_Eps + f_.diff(Sig).T * dot_Sig_Eps_).subs(dict(zip(Sig, Sig_)))

    # Combine the gradient of the Lagrangian and the rate of the flow potential
    f_Eps_ = f_.subs(dict(zip(Sig, Sig_)))
    R_g_ = dL_dEps_.row_insert(dL_dEps_.shape[0], sp.Matrix([f_Eps_]))

    return R_g_

In [ ]:
def derive_dg_dEps_(P, dot_lam):
    """
    Derive the evolution equations from the flow potential.
    """
    psi_, eps, dot_eps, Eps, dot_Eps, Sig, Sig_signs, f_, g_ = P.psi_, P.eps, P.dot_eps, P.Eps, P.dot_Eps, P.Sig, P.Sig_signs, P.f_, P.g_
    # Compute the gradient of the Lagrangian
    Sig_ = sp.simplify(sp.diag(*Sig_signs) * psi_.diff(Eps))
    dg_dEps_ = ((g_.diff(Sig).T * Sig_.jacobian(Eps)).T + g_.diff(Eps)).subs(dict(zip(Sig, Sig_)))
    return - sp.diag(*Sig_signs) * dg_dEps_ * dot_lam

### Example 1: R_g[ep] - Isotropic hardening plasticity

The specification of the constitutive model is done in terms of three functions. Here we exemplify the isotropic hardening plasticity in terms of 
$$
\psi_\mathrm{ep} := \frac{1}{2} E (\varepsilon - \varepsilon_\mathrm{p})^2 + \frac{1}{2} K z^2
$$ 
and the yield function
$$
 f_\mathrm{ep} := \left| {\sigma_\mathrm{p}} \right| - (f_\mathrm{c} + Z)
$$

In [ ]:
class P_ep:
    # external variables, internal variables, thermodynamic forces, and material parameters
    eps, eps_p, z = cymbols(r'\varepsilon \varepsilon_{\mathrm{p}} z', 'eps, eps_p, z', real=True)
    sig, sig_p, Z = cymbols(r'\sigma \sigma_{\mathrm{p}} Z', 'sig sig_p Z', real=True)
    E, K, f_c = cymbols('E K f_{\mathrm{c}}', 'E K f_c', positive=True)
    # free energy potential, yield function, and flow potential
    psi_ = sp.Rational(1, 2) * E * (eps - eps_p)**2 + sp.Rational(1, 2) * K * z**2
    f_ = sp.Abs(sig_p) - (f_c + Z)
    g_ = f_
    # collect the internal variables to enable generic derivation of the residuum
    Eps = sp.Matrix([eps_p, z])
    Sig = sp.Matrix([sig_p, Z])
    Sig_signs = [-1, 1]
    mparams = (E, K, f_c)
    # general functions - to be factored out to a base class?
    dot_Eps = sp.Matrix([sp.Symbol(name=f'\\dot{{{var.name}}}') for var in list(Eps)])
    dot_eps = sp.Symbol(name=f'\\dot{{{eps.name}}}')

# Access the attributes directly from the class
P_ep.psi_, P_ep.f_, P_ep.Eps, P_ep.dot_Eps, P_ep.Sig, P_ep.Sig_signs, P_ep.mparams


In [ ]:
R_gep_ = derive_R_g(P_ep, R_g.dot_lam)
R_gep_

In [ ]:
derive_dg_dEps_(P_ep, R_g.dot_lam)

#### Check the analytical solution

For isotropic hardening, this equation system can be solved explicitly - use it to check the symbolic scheme.

In [ ]:
dot_A_gep_ = P_ep.dot_Eps.row_insert(P_ep.dot_Eps.shape[0], sp.Matrix([[R_g.dot_lam]]))
sp.simplify(sp.solve(R_gep_, dot_A_gep_))

### Example 2: R_g[epd] Isotropic hardening plasticity and damage

The specification of the constitutive model is done in terms of three functions. Here we exemplify the isotropic hardening plasticity in terms of 
$$
\psi_\mathrm{} := \frac{1}{2} (1 - \omega) E (\varepsilon - \varepsilon_\mathrm{p})^2 + \frac{1}{2} K z^2,
$$ 
the yield function
$$
 f_\mathrm{epd} := \left| \frac{\sigma_\mathrm{p}}{1 - \omega} \right| - (f_\mathrm{c} + Z),
$$
and the flow potential
$$
 g_\mathrm{epd} = f + (1 - \omega) \frac{S}{r+1} \left( \frac{Y}{S} \right)^{r+1}
$$

In [ ]:
class P_epd:
    # external variables, internal variables, thermodynamic forces, and material parameters
    eps, eps_p, z, omega = cymbols(r'\varepsilon \varepsilon_\mathrm{p} z \omega', 
                                   'eps, eps_p, z, omega', real=True)
    sig, sig_p, Z, Y = cymbols('\sigma \sigma_\mathrm{p} Z Y', 
                               'sig sigma_p Z Y', real=True)
    E, K, f_c, S, r = cymbols('E K f_\mathrm{c} S r', 
                              'E K f_c S r', positive=True)
    # free energy potential, yield function, and flow potential
    psi_ = sp.Rational(1,2) * (1 - omega) * E * (eps - eps_p)**2 + sp.Rational(1,2) * K * z**2
    f_ = sp.Abs(sig_p / (1-omega)) - (f_c + Z)
    r = 1
    g_ = f_ + (1 - omega) * (S/(r+1)) * (Y/S)**(r+1)
    # collect the internal variables to enable generic derivation of the residuum
    Eps = sp.Matrix([eps_p, z, omega])
    Sig = sp.Matrix([sig_p, Z, Y])
    Sig_signs = [-1, 1, -1]
    mparams = (E, K, f_c, S)
    # general functions - to be factored out to a base class?
    dot_Eps = sp.Matrix([sp.Symbol(name=f'\\dot{{{var.name}}}') for var in list(Eps)])
    dot_eps = sp.Symbol(name=f'\\dot{{{eps.name}}}')
    
P_epd.psi_, P_epd.f_, P_epd.g_, P_epd.Eps, P_epd.dot_Eps, P_epd.Sig, P_epd.Sig_signs, P_epd.mparams

In [ ]:
R_gepd_ = derive_R_g(P_epd, R_g.dot_lam)
R_gepd_

In [ ]:
dot_A_gepd_ = P_epd.dot_Eps.row_insert(P_epd.dot_Eps.shape[0], sp.Matrix([[R_g.dot_lam]]))
dot_A_gepd_

In [ ]:
dot_A_gepd_solved_ = sp.solve(R_gepd_, dot_A_gepd_)
dot_A_gepd_solved_[dot_A_gepd_[2]]

## R_ig[.] - Parameterized incremental Residuum

In [ ]:
def derive_R_dR_n1(P, dot_lam):
    # time
    t = Cymbol(r't', codename='t')
    delta_t = Cymbol(r'\Delta t', codename='delta_t')
    
    # fundamental state
    Eps_n = sp.Matrix([Cymbol(f'{var.name}^{{(n)}}', codename='{var.codename}_n') for var in P.Eps])
    eps_n = Cymbol(r'\varepsilon^{(n)}', codename='eps_n')

    # increment
    delta_Eps = sp.Matrix([Cymbol(f'\\Delta {{{var.name}}}', codename='delta_{var.codename}') for var in P.Eps])
    delta_eps = Cymbol(r'\Delta {\varepsilon}', codename='delta_eps')
    delta_lam = Cymbol(r'\Delta {\lambda}', codename='delta_lam')

    # updated state
    Eps_n1 = Eps_n + delta_Eps
    eps_n1 = eps_n + delta_eps

    # rate of change
    dot_Eps_n = delta_Eps / delta_t
    dot_eps_n = delta_eps / delta_t
    dot_lam_n = delta_lam / delta_t

    # derive substitutions
    subs_dot_Eps = dict(zip(P.dot_Eps, dot_Eps_n))
    subs_dot_eps = {P.dot_eps: dot_eps_n}
    subs_dot_lam = {dot_lam: dot_lam_n}
    subs_Eps = dict(zip(P.Eps, Eps_n1))
    subs_eps = {P.eps: eps_n1}

    subs_n1 = {**subs_dot_Eps, **subs_dot_eps, **subs_dot_lam, **subs_Eps, **subs_eps}

    R_g = derive_R_g(P, dot_lam)
    # residuum vector in n+1 step
    R_n1 = R_g.subs(subs_n1)

    # construct the jacobian of the residuum
    delta_A = sp.Matrix([delta_Eps, delta_lam])
    dR_dA_n1_ = R_n1.jacobian(delta_A)
    dR_dA_n1 = dR_dA_n1_.replace(sp.Derivative, lambda *args: 0)

    return (eps_n, delta_eps, Eps_n, delta_A, delta_t), R_n1, dR_dA_n1


### R_ig[ep] - Residuum and Jacobian for elasto-plasticity with isotropic hardening

In [ ]:
(eps_n, delta_eps, Eps_n, delta_A, delta_t), R_ep_, dR_dA_ep_  = derive_R_dR_n1(P_ep, R_g.dot_lam)
R_ep_, dR_dA_ep_

In [ ]:
dR_dA_ep_.inverse() * R_ep_

In [ ]:
P_ep.mparams

In [ ]:
get_R_dR_n1_ep = sp.lambdify((eps_n, delta_eps, Eps_n, delta_A, delta_t, *P_ep.mparams), 
                            (R_ep_, dR_dA_ep_))
get_R_dR_n1_ep(1, 2, [0.3, 0.4], [0.5, 0.6, 0.0], 0.1, E=28000, K=0, f_c=30)

In [ ]:
import numpy as np
mparams_ep = dict(E=1, K=1, f_c=1)
delta_A_k = np.array([0.0, 0.0, 0.0])
delta_eps = 0.1
R_n1_epd_k, dR_n1_epd_k = get_R_dR_n1_ep(0, delta_eps, [0.0, 0.0], delta_A_k, 1, **mparams_ep)
delta_A_k += np.linalg.solve(dR_n1_epd_k, -R_n1_epd_k[:, 0])
R_n1_epd_k, dR_n1_epd_k = get_R_dR_n1_ep(0, delta_eps, [0.0, 0.0], delta_A_k, 1, **mparams_ep)
dR_n1_epd_k, R_n1_epd_k, delta_A_k

### R_ig[epd] - Residuum and Jacobian for elasto-plasticity with isotropic hardening and damage 

In [ ]:
(eps_n, delta_eps, Eps_n, delta_A, delta_t), R_epd_, dR_dA_epd_  = derive_R_dR_n1(P_epd, R_g.dot_lam)
R_epd_

In [ ]:
P_epd.mparams

In [ ]:
get_R_dR_n1_epd = sp.lambdify((eps_n, delta_eps, Eps_n, delta_A, delta_t, *P_epd.mparams), 
                            (R_epd_, dR_dA_epd_))


In [ ]:
import numpy as np
mparams_epd = dict(E=100, K=100, f_c=1, S=0.1)
delta_A_k = np.array([0.0, 0.0, 0.0, 0.0])
delta_eps = 1.1
Eps_0 = [0.0, 0.0, 0.0]
R_n1_epd_k, dR_n1_epd_k = get_R_dR_n1_epd(0, delta_eps, Eps_0, delta_A_k, 1, **mparams_epd)
delta_A_k += np.linalg.solve(dR_n1_epd_k, -R_n1_epd_k[:, 0])
# R_n1_epd_k, dR_n1_epd_k = get_R_dR_n1_epd(0, delta_eps, Eps_0, delta_A_k, 1, **mparams_epd)
# delta_A_k += np.linalg.solve(dR_n1_epd_k, -R_n1_epd_k[:, 0])
dR_n1_epd_k, -R_n1_epd_k, delta_A_k